In [ ]:
#---------
#  Recommend System in NetEase 
# 
#  Modify History : 2019 - Jan - 22 
#  Platform: Win7 + Python2 
#---------

### 1 从原始文件中抽取期望的歌单数据

In [2]:
#coding:urf-8
import json 
import sys

# 从Json文件中提取特定格式的文本数据 #
# 返回的文本格式：歌曲名字##歌曲标签##歌单ID##歌曲收藏数目 歌曲信息
def parse_song_inline(in_line):
    loaded_data = json.loads(in_line)
    name        = loaded_data['result']['name']  # 名字
    tags        = ",".join(loaded_data['result']['tags'])# 标签
    subscribed_count = loaded_data['result']['subscribedCount'] #  收藏数目
    
    if subscribed_count <= 100:
        #print "subscribed_count is less than 100, please check data ..."
        return False
    playlist_id = loaded_data['result']['id']  # 歌单ID
    tracks      =  loaded_data['result']['tracks'] # "tracks": [{歌曲1},{歌曲2}, ...]
    
    song_inforcontent = ''
    for track in tracks:
        try: # 歌曲信息中包含: => 歌单ID:::歌曲名字:::歌手名字:::歌曲流行度
            song_inforcontent +=   "\t" + ":::".join([str(track['id']),track['name'],track['artists'][0]['name'],str(track['popularity'])])
        except Exception,e:
            print "Exception information 1",e
            print track
            continue
    # 获取如下格式的文本信息
    # 格式：歌曲名字##歌曲标签##歌单ID##歌曲收藏数目 歌曲信息
    GotText =  name+"##"+tags+"##"+str(playlist_id)+'##'+str(subscribed_count)+song_inforcontent
    return GotText

# 将抽取的文本写入特定的text文件
def parse_files(input_file,out_file):
    outdata  = open(out_file,'w')
    for line in open(input_file):
        GotText = parse_song_inline(line)
        if GotText:
            outdata.write(GotText.encode('utf-8').strip()+"\n")
    outdata.close()

In [2]:
%time parse_files("./RawData/playlistdetail.all.json","./InternalData/1_music_playlist_info.txt")

Wall time: 15min 47s


### 2 将歌单数据处理成适用于推荐系统的数据格式

In [3]:
import surprise
import lightfm
# project  = offline modeleing + oneline prediction 

# 推荐机制：
# 1.针对用户推荐，每日推荐7 - 30首歌曲
# 2.针对你听到的歌曲，推荐类似的歌曲
# 3.针对你喜欢的歌手，推荐类似的歌手给用户

c:\python27\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [9]:
#coding:utf-8 
#为适应于surprise模型，将数据解析成userid itemid rating timestamp 行格式
import json
import sys

# check is null or not d
def is_null(s):
    return len(s.split(","))>2

def parse_song_inforcontent(song_inforcontent):
    try:
        song_id,name,artist,popularity = song_inforcontent.split(":::")
        return ",".join([song_id,"1.0",'1300000'])
        #return ",".join([song_id, name, artist, popularity])
    except Exception,e:
        #print "Exception information 2 ",e
        #print song_inforcontent
        return " "


def parse_playlist_inline(in_line):
    try:
        contents = in_line.strip().split("\t")  # strip 取出字符串首尾字符，默认是取出空格
        name,tags,playlist_id,subscribed_count = contents[0].split("##")
        songs_infor = map(lambda x: playlist_id +","+ parse_song_inforcontent(x),contents[1:])
        
        # filter (function,iterable):  将iterable 作用于function逐个判断，将满足条件的iterable  返回新list
        songs_information  = filter(is_null,songs_infor) 
        return "\n".join(songs_information)
    except Exception,e:
        #print "Exception information 3 ",e
        return False

# 将抽取的文本写入特定的text文件
def parse_files(input_file,out_file):
    outdata  = open(out_file,'w')
    for line in open(input_file):
        GotText = parse_playlist_inline(line)
        if GotText:
            outdata.write(GotText.encode('utf-8').strip()+"\n")
    outdata.close()

In [10]:
%time parse_files("./InternalData/1_music_playlist_info.txt","./InternalData/2_music_playlist_surpriseformat.txt")

Wall time: 6.52 s


### 处理流行歌单

In [11]:
%time parse_files("./RawData/popular.playlist","./InternalData/3_music_popularplaylist_surpriseformat.txt")

Wall time: 652 ms


### 3 保存歌曲以及歌单信息以备后面的建模使用

需要保存歌单ID => 歌单名字，以及歌曲ID =>歌曲名字 的信息以备用

In [23]:
#coding:utf-8
import cPickle as pickle
import sys 
def parse_playlist_get_infor(in_line,playlist_dic,song_dic):
    contents = in_line.strip().split("\t")
    name,tags,playlist_id,subscribed_count  = contents[0].split("##")
    playlist_dic[playlist_id] = name 
    for song in contents[1:]:
        try:
            song_id,song_name,artist,popularity = song.split(":::")
            song_dic[song_id] =  song_name + "\t" + artist
        except:
            print "FormatError.."
            print song  + "\n"

def parse_file(in_file,out_playlist,out_song):
    playlist_dic = {} # 歌单ID 到歌单的映射字典
    song_dic = {}     # 歌曲ID到歌曲名字的映射字典
    for line in open(in_file):
        parse_playlist_get_infor(line,playlist_dic,song_dic)
        
    #通过pickle 序列化保存成二进制的文件
    pickle.dump(playlist_dic,open(out_playlist,"wb"))
    
    # 通过playlist_doc = pickle.load(open("playlist.pkl","rb")) 重新载入
    pickle.dump(song_dic,open(out_song,"wb"))
    

In [24]:
%time parse_file("./InternalData/1_music_playlist_info.txt","./InternalData/4_playlist.pkl","./InternalData/5_song.pkl")

FormatError..
19169096:::

FormatError..
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

FormatError..
28935319:::

FormatError..
สั่น (Album Version):::Boy Sompob:::100.0

FormatError..
26902203:::What’s your name? (collaboration with 壇蜜)

FormatError..
:::SoulJa:::100.0

FormatError..
4967373:::テニスの王子様::ここからが・・・俺達II

FormatError..
:::V.A.:::5.0

FormatError..
28935319:::

FormatError..
สั่น (Album Version):::Boy Sompob:::100.0

FormatError..
376653:::野弧禅狂叱(宿香之战)

FormatError..
:::霹雳英雄:::5.0

FormatError..
374524:::赎?罪

FormatError..
赎罪岩:::霹雳英雄:::15.0

FormatError..
31563610:::

FormatError..
苍之礼赞:::花之祭P:::65.0

FormatError..
26902203:::What’s your name? (collaboration with 壇蜜)

FormatError..
:::SoulJa:::100.0

FormatError..
31838468:::Justiφ's(Tv Size Version)

FormatError..
:::佐藤和豊:::25.0

FormatError..
2924300:::

FormatError..
 Les Peulles Mortes (Autum Leaves):::Laura Fygi:::85.0

FormatError..
22642373:::

FormatError..
 FAIRY TAIL メインテーマ -Slow ver.-:::高梨康治:::95

FormatError..
2924300:::

FormatError..
 Les Peulles Mortes (Autum Leaves):::Laura Fygi:::85.0

FormatError..
28935319:::

FormatError..
สั่น (Album Version):::Boy Sompob:::100.0

FormatError..
27511866:::첫 사랑니 (Rum Pum Pum Pum)

FormatError..
:::f(x):::95.0

FormatError..
33054290:::

FormatError..
Heartbeats:::Dabin:::95.0

FormatError..
785061:::幻奏ノ滝

FormatError..
时音(フォールオブフォール~秋めく滝):::沙紗飛鳥:::60.0

FormatError..
27511866:::첫 사랑니 (Rum Pum Pum Pum)

FormatError..
:::f(x):::95.0

FormatError..
32272105:::

FormatError..
Wonderful Love (DJ Raf Remix):::Money Penny:::95.0

FormatError..
2924300:::

FormatError..
 Les Peulles Mortes (Autum Leaves):::Laura Fygi:::85.0

FormatError..
26878881:::

FormatError..
Wolfgang Amadeus Mozart: Serenata notturna in D, K.239 - 2. Menuetto - Trio:::Herbert von Karajan:::15.0

FormatError..
28935319:::

FormatError..
สั่น (Album Version):::Boy Sompob:::100.0

FormatError..
19169096:::

FormatError..
 Time to Say Goodbye (Con te partirò):::Sarah Brightm

FormatError..
17742280:::

FormatError..
Johann Sebastian Bach: Prelude and Fugue in C sharp (WTK, Book, No.3), BWV 848 - Prelude:::Friedrich Gulda:::25.0

FormatError..
31654791:::

FormatError..
European Intrigue:::Tim Wynn:::45.0

FormatError..
33054290:::

FormatError..
Heartbeats:::Dabin:::95.0

FormatError..
686282:::Chapter1 MainBGM 

FormatError..
PSP ver:::ARTERY VEIN:::55.0

FormatError..
26902203:::What’s your name? (collaboration with 壇蜜)

FormatError..
:::SoulJa:::100.0

FormatError..
33054290:::

FormatError..
Heartbeats:::Dabin:::95.0

FormatError..
401386160:::

FormatError..
 when you think about her and your brain says no don't do that:::In Love With A Ghost:::55.0

FormatError..
5090113:::Praia Brava歌手

FormatError..
Miraflores:::Various Artists:::10.0

FormatError..
725347:::fortissimo-the ultimate crisis- (instrumental)

FormatError..
:::fripSide:::50.0

FormatError..
22642373:::

FormatError..
 FAIRY TAIL メインテーマ -Slow ver.-:::高梨康治:::95.0

FormatError..
427373827::

FormatError..
401386160:::

FormatError..
 when you think about her and your brain says no don't do that:::In Love With A Ghost:::55.0

FormatError..
785061:::幻奏ノ滝

FormatError..
时音(フォールオブフォール~秋めく滝):::沙紗飛鳥:::65.0

FormatError..
27511866:::첫 사랑니 (Rum Pum Pum Pum)

FormatError..
:::f(x):::95.0

FormatError..
28935319:::

FormatError..
สั่น (Album Version):::Boy Sompob:::100.0

FormatError..
30474690:::

FormatError..
Life Of Sin pt.2:::MitiS:::70.0

FormatError..
28935319:::

FormatError..
สั่น (Album Version):::Boy Sompob:::100.0

FormatError..
27511866:::첫 사랑니 (Rum Pum Pum Pum)

FormatError..
:::f(x):::95.0

FormatError..
785061:::幻奏ノ滝

FormatError..
时音(フォールオブフォール~秋めく滝):::沙紗飛鳥:::65.0

FormatError..
33241159:::The End (Culture Code Remix)

FormatError..
:::Aruna:::20.0

FormatError..
686282:::Chapter1 MainBGM 

FormatError..
PSP ver:::ARTERY VEIN:::55.0

FormatError..
26902203:::What’s your name? (collaboration with 壇蜜)

FormatError..
:::SoulJa:::100.0

FormatError..
30474690:::

Format

In [25]:
%time parse_file("./RawData/popular.playlist","./InternalData/6_popular_playlist.pkl","./InternalData/7_popular_song.pkl")

FormatError..
405599088:::Make Them Wheels Roll

FormatError..
:::SAFIA:::100.0

FormatError..
424496188:::大王叫我来巡山 - （原唱：

FormatError..
 贾乃亮/贾云馨）:::流浪的蛙蛙:::65.0

FormatError..
19169096:::

FormatError..
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

FormatError..
26902203:::What’s your name? (collaboration with 壇蜜)

FormatError..
:::SoulJa:::100.0

FormatError..
33054290:::

FormatError..
Heartbeats:::Dabin:::95.0

FormatError..
427373827:::Champions (From "Hands of Stone") 

FormatError..
:::Usher:::30.0

FormatError..
31654811:::

FormatError..
American Cowboys:::Tim Wynn:::65.0

FormatError..
19169096:::

FormatError..
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

FormatError..
19169096:::

FormatError..
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

Wall time: 722 ms


### 4 使用协同过滤建模并进行预测
#### 使用surprise 自带的movielens 数据集

In [15]:
from surprise import SVD,KNNWithMeans
from surprise import Dataset
from surprise import evaluate,print_perf

# load movielens datasets
data = Dataset.load_builtin('ml-100k')

# k折交叉验证
%time data.split(n_folds = 3)

# 试试SVD分解
%time algo = SVD()
print (algo)
# Evaluate the performance of the algorithm on given data
# return A dictionary containing measures as keys and lists as values. Each list contains one entry per fold
%time perf = evaluate(algo,data,measures=[u'rmse', u'mae'] )
%time print_perf(perf)

Wall time: 68 ms
Wall time: 0 ns
Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1


c:\python27\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
c:\python27\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


RMSE: 0.9436
MAE:  0.7429
------------
Fold 2
RMSE: 0.9482
MAE:  0.7485
------------
Fold 3
RMSE: 0.9451
MAE:  0.7454
------------
------------
Mean RMSE: 0.9456
Mean MAE : 0.7456
------------
------------
Wall time: 21.4 s
        Fold 1  Fold 2  Fold 3  Mean    
MAE     0.7429  0.7485  0.7454  0.7456  
RMSE    0.9436  0.9482  0.9451  0.9456  
Wall time: 0 ns


In [20]:
data.raw_ratings[0]

(u'806', u'1514', 3.0, u'882385643')

### 算法调参

### 这里实现的算法是SVD 等，一些超参数的选择也会最终影响后果。下面可以使用sklearn 中的网格搜索交叉验证来GridsearchCV来选择最终的参数



In [19]:
from surprise import GridSearch
from surprise import SVD # or KNNWithMeans also can be OK!
from surprise import Dataset
# Following parameters use can be understand by help(GridSearch)
algo_class = SVD # KNNWithMeans also can be OK!
param_grid = {'n_epochs':[5,10], 'lr_all':[0.002,0.005], 'reg_all':[0.4,0.6]}
measures   = ['rmse', 'mae']

gridsearch = GridSearch(algo_class,param_grid,measures, n_jobs = -1 , verbose = True)
data       = Dataset.load_builtin('ml-100k')
data.split(n_folds = 4)

gridsearch.evaluate(data)

Running grid search for the following parameter combinations:
{u'lr_all': 0.002, u'reg_all': 0.4, u'n_epochs': 5}
{u'lr_all': 0.002, u'reg_all': 0.4, u'n_epochs': 10}
{u'lr_all': 0.002, u'reg_all': 0.6, u'n_epochs': 5}
{u'lr_all': 0.002, u'reg_all': 0.6, u'n_epochs': 10}
{u'lr_all': 0.005, u'reg_all': 0.4, u'n_epochs': 5}
{u'lr_all': 0.005, u'reg_all': 0.4, u'n_epochs': 10}
{u'lr_all': 0.005, u'reg_all': 0.6, u'n_epochs': 5}
{u'lr_all': 0.005, u'reg_all': 0.6, u'n_epochs': 10}
Resulsts:
{u'lr_all': 0.002, u'reg_all': 0.4, u'n_epochs': 5}
{u'MAE': 0.8018551244250034, u'RMSE': 0.9932174367810193}
----------
{u'lr_all': 0.002, u'reg_all': 0.4, u'n_epochs': 10}
{u'MAE': 0.7832613784600437, u'RMSE': 0.9749094382803938}
----------
{u'lr_all': 0.002, u'reg_all': 0.6, u'n_epochs': 5}
{u'MAE': 0.8110030936953079, u'RMSE': 0.9996999802164589}
----------
{u'lr_all': 0.002, u'reg_all': 0.6, u'n_epochs': 10}
{u'MAE': 0.7940102667564738, u'RMSE': 0.9834308452554275}
----------
{u'lr_all': 0.005, u'r

### 用户216 对电影11的打分是5.0分，时间是880234346

In [38]:
data.raw_ratings[1]  # user, item, rating, timestamp

(u'216', u'11', 5.0, u'880234346')

### 输出模型评价

In [12]:
gridsearch.best_score

CaseInsensitiveDefaultDict(list,
                           {'mae': 0.7712927572588933,
                            'rmse': 0.9623471906682217})

In [13]:
gridsearch.best_score['rmse']

0.9623471906682217

In [10]:
gridsearch.best_estimator

CaseInsensitiveDefaultDict(list,
                           {'mae': <surprise.prediction_algorithms.matrix_factorization.SVD at 0xfec4b38>,
                            'rmse': <surprise.prediction_algorithms.matrix_factorization.SVD at 0xfec4ef0>})

In [24]:
gridsearch.best_params

CaseInsensitiveDefaultDict(list,
                           {'mae': {'lr_all': 0.005,
                             'n_epochs': 10,
                             'reg_all': 0.4},
                            'rmse': {'lr_all': 0.005,
                             'n_epochs': 10,
                             'reg_all': 0.4}})

In [34]:
gridsearch.best_params['mae']

{'lr_all': 0.005, 'n_epochs': 10, 'reg_all': 0.4}

In [27]:
gridsearch.best_params['rmse']

{'lr_all': 0.005, 'n_epochs': 10, 'reg_all': 0.4}

In [11]:
gridsearch.best_score

CaseInsensitiveDefaultDict(list,
                           {'mae': 0.7712927572588933,
                            'rmse': 0.9623471906682217})

In [29]:
gridsearch.best_index

CaseInsensitiveDefaultDict(list, {'mae': 5, 'rmse': 5})

In [32]:
gridsearch.cv_results

defaultdict(list,
            {u'params': [{'lr_all': 0.002, 'n_epochs': 5, 'reg_all': 0.4},
              {'lr_all': 0.002, 'n_epochs': 10, 'reg_all': 0.4},
              {'lr_all': 0.002, 'n_epochs': 5, 'reg_all': 0.6},
              {'lr_all': 0.002, 'n_epochs': 10, 'reg_all': 0.6},
              {'lr_all': 0.005, 'n_epochs': 5, 'reg_all': 0.4},
              {'lr_all': 0.005, 'n_epochs': 10, 'reg_all': 0.4},
              {'lr_all': 0.005, 'n_epochs': 5, 'reg_all': 0.6},
              {'lr_all': 0.005, 'n_epochs': 10, 'reg_all': 0.6}],
             u'scores': [{'MAE': 0.8020509797391934,
               'RMSE': 0.9934170477955387},
              {'MAE': 0.783339322044289, 'RMSE': 0.9752593213784541},
              {'MAE': 0.8108648660726613, 'RMSE': 0.9997787387938837},
              {'MAE': 0.7942050010530397, 'RMSE': 0.9837870959953469},
              {'MAE': 0.7797664765655643, 'RMSE': 0.971444140579676},
              {'MAE': 0.7712927572588933, 'RMSE': 0.9623471906682217},
    

In [51]:
help(GridSearch)

Help on class GridSearch in module surprise.evaluate:

class GridSearch
 |  .. warning::
 |      Deprecated since version 1.05. Use :func:`GridSearchCV
 |      <surprise.model_selection.search.GridSearchCV>` instead. This
 |      class will be removed in later versions.
 |  
 |  The :class:`GridSearch` class, used to evaluate the performance of an
 |  algorithm on various combinations of parameters, and extract the best
 |  combination. It is analogous to `GridSearchCV
 |  <http://scikit-learn.org/stable/modules/generated/sklearn.
 |  model_selection.GridSearchCV.html>`_ from scikit-learn.
 |  
 |  See :ref:`User Guide <tuning_algorithm_parameters>` for usage.
 |  
 |  Args:
 |      algo_class(:obj:`AlgoBase             <surprise.prediction_algorithms.algo_base.AlgoBase>`): The class
 |          object of the algorithm to evaluate.
 |      param_grid(dict): Dictionary with algorithm parameters as keys and
 |          list of values as keys. All combinations will be evaluated with
 |   

## 协同过滤建模后，根据一个items寻找相似度最高的item,使用algo.get_neighbors 函数
 

In [1]:
### 先认识一下处理的数据 特征
### in C:\Users\Yazhou\.surprise_data\ml-100k\ml-100k\u/item
import io
import os
file_name   = (os.path.expanduser('~') + '/.surprise_data/ml-100k/ml-100k/u.item')
with io.open(file_name,'r',encoding = 'ISO-8859-1') as f:
    for line in f:
        #print ">>>Output: ",line
        pass

### 1. 针对自带的Movielens 数据集做推荐

In [2]:
from __future__ import (absolute_import,division,print_function,unicode_literals)
import os 
import io
from surprise import Dataset
from surprise import KNNBasic # or KNNBaseline

file_name   = (os.path.expanduser('~') + '/.surprise_data/ml-100k/ml-100k/u.item')
def read_item_names():
    "获取电影ID到电影名字，电影名字到电影ID的映射关系"
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name,'r',encoding = 'ISO-8859-1') as f:   #  "ISO-8859-1" 是官方指定的字体
        for line in f:
            line = line.split('|')  # 从上面的数据格式可以看出原始数据是ID|Name 的先后顺序
            #print("line is:",line)
            rid_to_name[line[0]] = line[1]  # 字典，line[0] is key, line[1] is value # 数据在dict中的位置并不是line[0]
            name_to_rid[line[1]] = line[0]  # 字典, line[1] is key, line[0] is value,
            #print ("line[0] is ", line[0])
            #print ("line[1] is ", line[1])
            #print (">>> rid_to_name is",rid_to_name)
            #print ("<<< name_to_rid is",name_to_rid)
    return rid_to_name,name_to_rid  

"利用算法来计算item之间的相似度"
data = Dataset.load_builtin('ml-100k')
import random
my_seed = 200
random.seed(my_seed)
trainset = data.build_full_trainset()
# data.split(n_folds = 4,shuffle = True) # Split the dataset into folds for future cross-validation from help(data)
sim_options = {'name':'pearson_baseline','user_based':False,'verbose':True}
# 基于用户的协同过滤
algo = KNNBasic(sim_options = sim_options)
algo.fit(trainset)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [12]:
"下面的例子证明字典里面的数据：'This is one new movies' 并没有出现在第3的位置"
dict1 = {u'1': u'Toy Story (1995)', u'2': u'GoldenEye (1995)'}
dict1[3] = 'This is one new movies'
dict1

{3: u'This is one new movies',
 u'1': u'Toy Story (1995)',
 u'2': u'GoldenEye (1995)'}

In [2]:
"计算pearson_baseline相似矩阵"
similarity_matrix = algo.compute_similarities()

Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [3]:
"获取电影ID到电影名字，电影名字到电影ID的数据映射关系"
rid_to_name,name_to_rid   = read_item_names()

In [4]:
"rid_to_name 数据:电影ID到电影名字的映射字典"
rid_to_name

{u'344': u'Apostle, The (1997)',
 u'345': u'Deconstructing Harry (1997)',
 u'346': u'Jackie Brown (1997)',
 u'347': u'Wag the Dog (1997)',
 u'340': u'Boogie Nights (1997)',
 u'341': u'Critical Care (1997)',
 u'342': u'Man Who Knew Too Little, The (1997)',
 u'343': u'Alien: Resurrection (1997)',
 u'348': u'Desperate Measures (1998)',
 u'349': u'Hard Rain (1998)',
 u'1653': u'Entertaining Angels: The Dorothy Day Story (1996)',
 u'298': u'Face/Off (1997)',
 u'299': u'Hoodlum (1997)',
 u'296': u'Promesse, La (1996)',
 u'297': u"Ulee's Gold (1997)",
 u'294': u'Liar Liar (1997)',
 u'295': u'Breakdown (1997)',
 u'292': u'Rosewood (1997)',
 u'293': u'Donnie Brasco (1997)',
 u'290': u'Fierce Creatures (1997)',
 u'291': u'Absolute Power (1997)',
 u'270': u'Gattaca (1997)',
 u'271': u'Starship Troopers (1997)',
 u'272': u'Good Will Hunting (1997)',
 u'273': u'Heat (1995)',
 u'274': u'Sabrina (1995)',
 u'275': u'Sense and Sensibility (1995)',
 u'276': u'Leaving Las Vegas (1995)',
 u'277': u'Restor

In [5]:
"name_to_rid 数据:电影名字到电影ID的映射字典"
name_to_rid

{u'Adventures of Pinocchio, The (1996)': u'1060',
 u'To Cross the Rubicon (1991)': u'1564',
 u'Birdcage, The (1996)': u'25',
 u'B. Monkey (1998)': u'1679',
 u'Little Big League (1994)': u'1032',
 u'Magnificent Seven, The (1954)': u'510',
 u'Chairman of the Board (1998)': u'1654',
 u'Mortal Kombat (1995)': u'541',
 u'Tin Men (1987)': u'712',
 u'Endless Summer 2, The (1994)': u'1184',
 u'Village of the Damned (1995)': u'565',
 u'Cat on a Hot Tin Roof (1958)': u'499',
 u'SubUrbia (1997)': u'1428',
 u'Air Bud (1997)': u'261',
 u'Johnny 100 Pesos (1993)': u'1365',
 u'Bye Bye, Love (1995)': u'1446',
 u'Men With Guns (1997)': u'1646',
 u'Beautiful Thing (1996)': u'1137',
 u'2 Days in the Valley (1996)': u'1011',
 u'Jerky Boys, The (1994)': u'1484',
 u'Close Shave, A (1995)': u'408',
 u'Amadeus (1984)': u'191',
 u'Critical Care (1997)': u'341',
 u'Clockwork Orange, A (1971)': u'179',
 u'Country Life (1994)': u'1290',
 u'Boys in Venice (1996)': u'1359',
 u'Wings of Courage (1995)': u'1515',
 u'

In [55]:
"原始电影ID到 inner id 的转化过程: 转化来着模型的数据需要"
raw_movie_id  =  name_to_rid['Bye Bye, Love (1995)']  # Toy Story (1995) #  Bye Bye, Love (1995)
#raw_movie_id  => 1446 raw movie id 

iinder_movie_id = algo.trainset.to_inner_iid(raw_movie_id)
#iinder_movie_id => 1217 inner movie id for using in model

In [56]:
"通过get_neighbors 来计算inner id: iid = 1217 的最近/最相似的k个相似"
iid = iinder_movie_id
k   = 12

"找到的距离目标最近的几个item，返回的inner id:iid" "to_raw_iid/to_inner_iid 都是针对item的id转换" 
kneighbors_iid   = algo.get_neighbors(iid, k)  # Returns:The list of the ``k`` (inner) ids of the closest users (or items)to ``iid``

"将inner id:iid 转换成原始id:rawid"
kneighbors_rawid = (algo.trainset.to_raw_iid(k_iid) for k_iid in kneighbors_iid)  #  返回生成器

"依照原始id:rawid 找到对应的电影名字"
movie_withrawid = [rid_to_name[k_rawid] for k_rawid in kneighbors_rawid] # 也可以是生成器，但是写成了list了

"打印输出跟目标最相似的几个电影名字"
#for movie in movie_withrawid:
#    print ("找到跟目标电影最相似的k(k = 12)个电影分别是:",movie)

for kth in range(0,k):
    print ("找到跟目标电影最相似的第 %d 电影是: %s" %(1 + kth,movie_withrawid[kth]))
    

找到跟目标电影最相似的第 1 电影是: Paper, The (1994)
找到跟目标电影最相似的第 2 电影是: Godfather, The (1972)
找到跟目标电影最相似的第 3 电影是: Hunt for Red October, The (1990)
找到跟目标电影最相似的第 4 电影是: Down Periscope (1996)
找到跟目标电影最相似的第 5 电影是: Brady Bunch Movie, The (1995)
找到跟目标电影最相似的第 6 电影是: Speed (1994)
找到跟目标电影最相似的第 7 电影是: Home Alone (1990)
找到跟目标电影最相似的第 8 电影是: Young Frankenstein (1974)
找到跟目标电影最相似的第 9 电影是: Fifth Element, The (1997)
找到跟目标电影最相似的第 10 电影是: That Thing You Do! (1996)
找到跟目标电影最相似的第 11 电影是: Willy Wonka and the Chocolate Factory (1971)
找到跟目标电影最相似的第 12 电影是: With Honors (1994)


### 2.使用音乐数据预测

In [1]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
import os 
import io
from surprise import KNNBaseline,Reader,KNNBasic,KNNWithMeans
from surprise import Dataset
import cPickle as pickle

"重建流行歌单ID到歌单名字的映射关系"
playlistid_to_name = {}
playlistid_to_name =  pickle.load(open("./InternalData/6_popular_playlist.pkl", 'rb'))
print("重建流行歌单ID到歌单名字的映射关系 完成....")

"重建歌单名到歌单ID的映射关系"
playlistname_to_id =  {}
for playlist_id in playlistid_to_name:
    playlistname_to_id[playlistid_to_name[playlist_id]] =  playlist_id
print("重建流行歌单名到歌单ID的映射关系 完成....")

file_path = os.path.expanduser("./InternalData/3_music_popularplaylist_surpriseformat.txt")

"指定文件格式..."
reader = Reader(line_format = "user item rating timestamp",sep = ',')

"载入指定的数据集"
loaded_musicdata = Dataset.load_from_file(file_path = file_path, reader = reader)

"计算歌曲之间的相识度"
music_full_trainset = loaded_musicdata.build_full_trainset()
print("构建数据集合完成。。。")

重建流行歌单ID到歌单名字的映射关系 完成....
重建流行歌单名到歌单ID的映射关系 完成....
构建数据集合完成。。。


In [5]:
"歌单ID到歌单的映射关系，存放在这样一个字典中"
#playlistid_to_name.keys()
#playlistid_to_name.values()

u'\u6b4c\u5355ID\u5230\u6b4c\u5355\u7684\u6620\u5c04\u5173\u7cfb\uff0c\u5b58\u653e\u5728\u8fd9\u6837\u4e00\u4e2a\u5b57\u5178\u4e2d'

In [6]:
print (playlistid_to_name[playlistid_to_name.keys()[2] ])

100种深情皆苦 | 你又不知道我难过


In [7]:
"music 基本数据特征"
music_full_trainset.n_users
#music_full_trainset.n_items
#music_full_trainset.rating_scale
#music_full_trainset.global_mean

1076

### 2.1下面是针对歌单的个性推荐

In [2]:
"开始训练模型 - 找到相似的歌单(默认一个用户只建立一个歌单playlist)"
algo = KNNBaseline()  # "默认是用户base 的KNN"
algo.train(music_full_trainset)

"目标歌单名字"
target_playlistname  = playlistname_to_id.keys()[39]
print ("目标歌单名字:",target_playlistname)

"目标歌单ID"
target_playlistid   = playlistname_to_id[target_playlistname]
print ("目标歌单原始ID:",target_playlistid)

"将原始歌单ID转成inner id,用于get_neighbors 预测" ".to_inner_uid/to_raw_uid 都是针对user:uid的函数推荐"
target_playlistiid  = algo.trainset.to_inner_uid(target_playlistid)
print ("目标歌单内部ID:",target_playlistiid)
k =  10
playlist_neignhbors_kid = algo.get_neighbors(target_playlistiid,k = k)

"将预测出的k个inner id 转成原始id:raw_id"
playlist_neignhbors_rid =  [algo.trainset.to_raw_uid(innerid) for innerid in playlist_neignhbors_kid]
"有k 个原始id:raw_id 找到对应的歌单名字"
playlist_neignhbors_name = [playlistid_to_name[raw_uid] for raw_uid in playlist_neignhbors_rid]

"打印输出这k个电影名字:"
print("\n跟目标歌单<<",target_playlistname,">>相似的k个歌单分别是:" )
for raw_playlistname in playlist_neignhbors_name:
    print (raw_playlistname,"对应的inner id是:",algo.trainset.to_inner_uid(playlistname_to_id[raw_playlistname]))

c:\python27\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
目标歌单名字: 世事无常，唯愿你好
目标歌单原始ID: 306948578
目标歌单内部ID: 427

跟目标歌单<< 世事无常，唯愿你好 >>相似的k个歌单分别是:
【华语】暖心物语 纯白思念 对应的inner id是: 3
暗暗作祟| 不甘朋友不敢恋人 对应的inner id是: 15
专属你的周杰伦 对应的inner id是: 18
「华语歌曲」 对应的inner id是: 23
[小风收集]21世纪年轻人的音乐 对应的inner id是: 24
十七岁那年，以为能和你永远 对应的inner id是: 28
热门流行华语歌曲50首 对应的inner id是: 31
最易上手吉他弹唱超精选 对应的inner id是: 40
打开任意门，就有对的人 对应的inner id是: 42
行车路上，一曲长歌 对应的inner id是: 45


In [5]:
"针对歌曲个性推荐"
from __future__ import (absolute_import, division, print_function,unicode_literals)
import os 
import io
from surprise import KNNBaseline,Reader,KNNBasic,KNNWithMeans
from surprise import Dataset
import cPickle as pickle

"重建流行歌曲ID到歌曲名字的映射关系"
songid_to_name = {}
songid_to_name =  pickle.load(open("./InternalData/7_popular_song.pkl", 'rb'))
print ("歌曲ID到歌曲名字的映射...完成！")
"歌曲名字到歌曲ID的映射"
songname_to_id = {}
for song_id in songid_to_name:
    songname_to_id[songid_to_name[song_id]]= song_id # 名字做key, id做value
print("歌曲名字到歌曲ID的映射....完成！")

file_path = os.path.expanduser("./InternalData/3_music_popularplaylist_surpriseformat.txt")

"指定文件格式..."
reader = Reader(line_format = "user item rating timestamp",sep = ',')

"载入指定的数据集"
loaded_songdata    = Dataset.load_from_file(file_path = file_path, reader = reader)
song_full_trainset = loaded_songdata.build_full_trainset()

"模型"
algo = KNNWithMeans()
algo.train(song_full_trainset)

"目标歌曲名字"
target_songname =  songname_to_id.keys()[98]
print("目标歌曲名字 是:",target_songname)
target_songid  = songname_to_id[target_songname]
algo.train.to-




歌曲ID到歌曲名字的映射...完成！
歌曲名字到歌曲ID的映射....完成！
Computing the msd similarity matrix...
Done computing similarity matrix.
目标歌曲名字 是: 难忘的一天	许巍


### 2.2 下面是针对歌曲的个性推荐

In [3]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
import os 
import io
from surprise import KNNBaseline,Reader,KNNBasic,KNNWithMeans
from surprise import Dataset
import cPickle as pickle

"重建流行歌曲ID到歌曲名字的映射关系"
songid_to_name = {}
songid_to_name =  pickle.load(open("./InternalData/7_popular_song.pkl", 'rb'))
print ("歌曲ID到歌曲名字的映射...完成！")
"歌曲名字到歌曲ID的映射"
songname_to_id = {}
for song_id in songid_to_name:
    songname_to_id[songid_to_name[song_id]]= song_id # 名字做key, id做value
print("歌曲名字到歌曲ID的映射....完成！")

"inner id = 4 的用户"
user_inner_id =  4

"ur(:obj:`defaultdict` of :obj:`list`): The users ratings. This is a \
dictionary containing lists of tuples of the form ``(item_inner_id,\
rating)``. The keys are user inner ids."
item_inner_id_andrating = music_full_trainset.ur[user_inner_id]  # user_rating 返回的是:(item_inner_id,rating)
item_inner_id = map(lambda x:x[0], item_inner_id_andrating)

for item_iid in item_inner_id:
    print (algo.predict(uid = user_inner_id,iid = item_iid,r_ui = None, clip = True,verbose=False),
           songid_to_name[algo.trainset.to_raw_iid(item_iid)])

歌曲ID到歌曲名字的映射...完成！
歌曲名字到歌曲ID的映射....完成！
user: 4          item: 478        r_ui = None   est = 1.00   {u'was_impossible': False} 听见下雨的声音	魏如昀
user: 4          item: 429        r_ui = None   est = 1.00   {u'was_impossible': False} 梦一场	萧敬腾
user: 4          item: 936        r_ui = None   est = 1.00   {u'was_impossible': False} 干杯	西瓜Kune
user: 4          item: 937        r_ui = None   est = 1.00   {u'was_impossible': False} 给自己的歌 (Live) - live	纵贯线
user: 4          item: 938        r_ui = None   est = 1.00   {u'was_impossible': False} 小半	陈粒
user: 4          item: 939        r_ui = None   est = 1.00   {u'was_impossible': False} 思念是一种病(Live) - live	张震岳
user: 4          item: 940        r_ui = None   est = 1.00   {u'was_impossible': False} 可以不可以	丁当
user: 4          item: 941        r_ui = None   est = 1.00   {u'was_impossible': False} 秋酿	房东的猫
user: 4          item: 616        r_ui = None   est = 1.00   {u'was_impossible': False} 退后	周杰伦
user: 4          item: 942        r_ui = None   est = 1.00   

### 使用矩阵分解做预测

In [4]:
### 使用NMF
from surprise import NMF,evaluate 
from surprise import Dataset

file_path = os.path.expanduser("./InternalData/3_music_popularplaylist_surpriseformat.txt")
"指定文件格式..."
reader = Reader(line_format = "user item rating timestamp",sep = ',')

"载入指定的数据集"
loaded_musicdata = Dataset.load_from_file(file_path = file_path, reader = reader)

"计算歌曲之间的相识度"
trainset = loaded_musicdata.build_full_trainset()
print("构建数据集合完成。。。")

"建模"
algo= NMF()
algo.train(trainset)
print ("建模完成。。。")

"指定 user inner id = 4"
user_inner_id = 4 
item_inner_id_andrating = trainset.ur[user_inner_id]  # user_rating 返回的是:(item_inner_id,rating)
item_inner_id = map(lambda x:x[0], item_inner_id_andrating)

"这里不同的是讲inner id全部转成 raw user and itme id 做predict "
for item_iid in item_inner_id:
    print (algo.predict(uid = algo.trainset.to_raw_uid(user_inner_id),iid = algo.trainset.to_raw_iid(item_iid),r_ui = 1, clip = True,verbose=False),
           songid_to_name[algo.trainset.to_raw_iid(item_iid)])

构建数据集合完成。。。
建模完成。。。
user: 400232387  item: 27724082   r_ui = 1.00   est = 1.00   {u'was_impossible': False} 听见下雨的声音	魏如昀
user: 400232387  item: 167916     r_ui = 1.00   est = 1.00   {u'was_impossible': False} 梦一场	萧敬腾
user: 400232387  item: 408307325  r_ui = 1.00   est = 1.00   {u'was_impossible': False} 干杯	西瓜Kune
user: 400232387  item: 394618     r_ui = 1.00   est = 1.00   {u'was_impossible': False} 给自己的歌 (Live) - live	纵贯线
user: 400232387  item: 421423806  r_ui = 1.00   est = 1.00   {u'was_impossible': False} 小半	陈粒
user: 400232387  item: 394485     r_ui = 1.00   est = 1.00   {u'was_impossible': False} 思念是一种病(Live) - live	张震岳
user: 400232387  item: 5239563    r_ui = 1.00   est = 1.00   {u'was_impossible': False} 可以不可以	丁当
user: 400232387  item: 30635613   r_ui = 1.00   est = 1.00   {u'was_impossible': False} 秋酿	房东的猫
user: 400232387  item: 185884     r_ui = 1.00   est = 1.00   {u'was_impossible': False} 退后	周杰伦
user: 400232387  item: 276936     r_ui = 1.00   est = 1.00   {u'was_impossible':

### 可以使用以下方法来存储和重新加载模型

In [45]:
import surprise
surprise.dump.dump("./InternalData/Recommendation.model",algo = algo) # save to local

"Reload Model Again"
reload_algo =  surprise.dump.load("./InternalData/Recommendation.model")

### 不同推荐系统算法之间的差距评估

In [7]:
"首先载入数据Dataset"
from surprise import NMF,evaluate,NormalPredictor,BaselineOnly,KNNBasic,KNNWithMeans,KNNBaseline,SVD 
from surprise import Dataset,Reader,SVDpp,SlopeOne,CoClustering
import os,io
file_path = os.path.expanduser("./InternalData/3_music_popularplaylist_surpriseformat.txt")
"指定文件格式..."
reader = Reader(line_format = "user item rating timestamp",sep = ',')

"载入指定的数据集"
loaded_musicdata = Dataset.load_from_file(file_path = file_path, reader = reader)
loaded_musicdata.split(n_folds = 5)

"Return a list of ratings (user, item, rating, timestamp) read from\ file_name"
#loaded_musicdata.read_ratings(file_path)


u'Return a list of ratings (user, item, rating, timestamp) read from\\ file_name'

In [40]:
"Evaluate the performance of the algorithm on given data"
algo = NMF()
perf = evaluate(algo, data = loaded_musicdata, measures=[u'rmse',u'mae'], with_dump=False, dump_dir=None, verbose=1)
print perf

Evaluating RMSE, MAE of algorithm NMF.

------------
Fold 1
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------
defaultdict(<type 'list'>, {u'mae': [0.0, 0.0, 0.0, 0.0, 0.0], u'rmse': [0.0, 0.0, 0.0, 0.0, 0.0]})


In [41]:
"Evaluate the performance of the algorithm on given data"
algo = NormalPredictor()
perf = evaluate(algo, data = loaded_musicdata, measures=[u'rmse',u'mae'], with_dump=False, dump_dir=None, verbose=1)
print perf

Evaluating RMSE, MAE of algorithm NormalPredictor.

------------
Fold 1
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------
defaultdict(<type 'list'>, {u'mae': [0.0, 0.0, 0.0, 0.0, 0.0], u'rmse': [0.0, 0.0, 0.0, 0.0, 0.0]})


In [42]:
"Evaluate the performance of the algorithm on given data"
algo = BaselineOnly()
perf = evaluate(algo, data = loaded_musicdata, measures=[u'rmse',u'mae'], with_dump=False, dump_dir=None, verbose=1)
print perf

Evaluating RMSE, MAE of algorithm BaselineOnly.

------------
Fold 1
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
Estimating biases using als...
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------
defaultdict(<type 'list'>, {u'mae': [0.0, 0.0, 0.0, 0.0, 0.0], u'rmse': [0.0, 0.0, 0.0, 0.0, 0.0]})


In [43]:
"Evaluate the performance of the algorithm on given data"
algo = KNNBasic()
perf = evaluate(algo, data = loaded_musicdata, measures=[u'rmse',u'mae'], with_dump=False, dump_dir=None, verbose=1)
print perf

Evaluating RMSE, MAE of algorithm KNNBasic.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------
defaultdict(<type 'list'>, {u'mae': [0.0, 0.0, 0.0, 0.0, 0.0], u'rmse': [0.0, 0.0, 0.0, 0.0, 0.0]})


In [44]:
"Evaluate the performance of the algorithm on given data"
algo = KNNWithMeans()
perf = evaluate(algo, data = loaded_musicdata, measures=[u'rmse',u'mae'], with_dump=False, dump_dir=None, verbose=1)
print perf

Evaluating RMSE, MAE of algorithm KNNWithMeans.

------------
Fold 1
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 2
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 3
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 4
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
Fold 5
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
MAE:  0.0000
------------
------------
Mean RMSE: 0.0000
Mean MAE : 0.0000
------------
------------
defaultdict(<type 'list'>, {u'mae': [0.0, 0.0, 0.0, 0.0, 0.0], u'rmse': [0.0, 0.0, 0.0, 0.0, 0.0]})


In [45]:
"Evaluate the performance of the algorithm on given data"
algo = SVD()
perf = evaluate(algo, data = loaded_musicdata, measures=[u'rmse',u'mae'], with_dump=False, dump_dir=None, verbose=1)
print perf

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.0365
MAE:  0.0166
------------
Fold 2
RMSE: 0.0365
MAE:  0.0166
------------
Fold 3
RMSE: 0.0365
MAE:  0.0166
------------
Fold 4
RMSE: 0.0366
MAE:  0.0167
------------
Fold 5
RMSE: 0.0367
MAE:  0.0168
------------
------------
Mean RMSE: 0.0366
Mean MAE : 0.0167
------------
------------
defaultdict(<type 'list'>, {u'mae': [0.01662157364728032, 0.016584225356639986, 0.016605750311511524, 0.016728002266939445, 0.016829999107087646], u'rmse': [0.03646898490849693, 0.0365140019773, 0.036506678247635156, 0.03664450724028625, 0.0367032737619232]})


In [46]:
"Evaluate the performance of the algorithm on given data"
algo = SVDpp()  # SVD++
perf = evaluate(algo, data = loaded_musicdata, measures=[u'rmse','mae'], with_dump=False, dump_dir=None, verbose=1)
print perf

Evaluating RMSE, MAE of algorithm SVDpp.

------------
Fold 1
RMSE: 0.0147
MAE:  0.0063
------------
Fold 2
RMSE: 0.0147
MAE:  0.0063
------------
Fold 3
RMSE: 0.0149
MAE:  0.0065
------------
Fold 4
RMSE: 0.0151
MAE:  0.0065
------------
Fold 5
RMSE: 0.0153
MAE:  0.0066
------------
------------
Mean RMSE: 0.0149
Mean MAE : 0.0064
------------
------------
defaultdict(<type 'list'>, {'mae': [0.006321437663896232, 0.006263019339022795, 0.006458306290878363, 0.0065106171361254085, 0.006585042253559304], u'rmse': [0.014696295027144698, 0.014728653951463349, 0.014901658710573053, 0.015100227273104618, 0.015313478223964816]})


In [13]:
"Evaluate the performance of the algorithm on given data"
algo = CoClustering()
perf = evaluate(algo, data = loaded_musicdata, measures=[u'rmse'], with_dump=False, dump_dir=None, verbose=1)
print (perf)


Evaluating RMSE of algorithm CoClustering.

------------
Fold 1
RMSE: 0.0000
------------
Fold 2
RMSE: 0.0000
------------
Fold 3
RMSE: 0.0000
------------
Fold 4
RMSE: 0.0000
------------
Fold 5
RMSE: 0.0000
------------
------------
Mean RMSE: 0.0000
------------
------------
defaultdict(<type 'list'>, {u'rmse': [0.0, 0.0, 0.0, 0.0, 0.0]})


In [1]:
#help(data)
#help(algo)
#help(KNNBaseline)
#help(KNNWithMeans)
#help(Reader)
#help(Dataset)
#help(music_full_trainset)

In [47]:
#rid_to_name,name_to_rid = read_item_names()
#rid_to_name['344']

In [48]:
#name_to_rid.values()
#name_to_rid.keys()
#name_to_rid["To Cross the Rubicon (1991)"]

{}